In [8]:
import sys
import time
import numpy as np
import pandas as pd
from gensim.models import KeyedVectors
from sklearn.metrics import accuracy_score
from __future__ import division
from collections import Counter

In [9]:
wv = KeyedVectors.load('embs_train.kv')

In [10]:
def sentence_embedding(sentence, word_counts):
    tokens = sentence.split()
    embeddings = []
    for token in tokens:
        if token in wv and word_counts[token] > 1:
            embeddings.append(wv[token])
    if len(embeddings) > 0:
        return np.mean(embeddings, axis=0)
    else:
        return np.zeros(wv.vector_size)

In [11]:
def read_from(dataframe):
    for i, row in dataframe.iterrows():
        label = 1 if row['Label'] == '+' else -1
        yield (label, row['Review_Embedding'])

In [12]:
def test(dev_data, model):
    tot, err = 0, 0
    for label, emb in read_from(dev_data):
        err += label * (np.dot(model, emb)) <= 0
    return err / len(dev_data)

In [13]:
def averaged_perceptron(train_data, dev_data, epochs=10):
    t = time.time()
    best_err = 1.
    avg_model = np.zeros(train_data['Review_Embedding'].iloc[0].shape)  
    model = np.zeros(train_data['Review_Embedding'].iloc[0].shape) 

    
    word_counts = Counter()
    for review in train_data['Review']:
        word_counts.update(review.split())

    for it in range(1, epochs + 1):
        updates = 0
        for label, emb in read_from(train_data):
            if label * (np.dot(model, emb)) <= 0:
                updates += 1
                model += label * emb
            avg_model += model

        dev_err = test(dev_data, avg_model)
        best_err = min(best_err, dev_err)
        print("epoch %d, update %.1f%%, dev %.1f%%" % (it, updates / len(train_data) * 100, dev_err * 100))
    
    avg_model /= (epochs * len(train_data))  
    print("best dev err %.1f%%, time: %.1f secs" % (best_err * 100, time.time() - t))


In [14]:
if __name__ == "__main__":
    train_data = pd.read_csv('C:/3rd term/ML/HW-4/hw4-data/train.txt', sep='\t', header=None, names=['Label', 'Review'])
    dev_data = pd.read_csv('C:/3rd term/ML/HW-4/hw4-data/dev.txt', sep='\t', header=None, names=['Label', 'Review'])
    
    
    word_counts = Counter()
    for review in train_data['Review']:
        word_counts.update(review.split())

    
    train_data['Review_Embedding'] = train_data['Review'].apply(lambda x: sentence_embedding(x, word_counts))
    dev_data['Review_Embedding'] = dev_data['Review'].apply(lambda x: sentence_embedding(x, word_counts))
    
    averaged_perceptron(train_data, dev_data, epochs=10)

epoch 1, update 32.0%, dev 25.0%
epoch 2, update 30.4%, dev 25.1%
epoch 3, update 30.3%, dev 24.4%
epoch 4, update 29.9%, dev 24.2%
epoch 5, update 30.4%, dev 24.2%
epoch 6, update 30.7%, dev 24.5%
epoch 7, update 30.4%, dev 24.3%
epoch 8, update 30.1%, dev 24.5%
epoch 9, update 29.9%, dev 24.7%
epoch 10, update 30.2%, dev 24.7%
best dev err 24.2%, time: 5.0 secs
